In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
import os
import csv
import joblib
import pywt
import random
import numpy as np
import cv2 as cv
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from scipy.fftpack import dct

In [3]:
#for binarization
def binary_otsus(image, filter:int=1):
    if len(image.shape) == 3:
        gray_img = cv.cvtColor(image, cv.COLOR_BGR2GRAY)
    else:
        gray_img = image

    # Otsus Binarization
    if filter != 0:
        blur = cv.GaussianBlur(gray_img, (3,3), 0)
        binary_img = cv.threshold(blur, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1]
    else:
        binary_img = cv.threshold(gray_img, 0, 255, cv.THRESH_BINARY+cv.THRESH_OTSU)[1]

    # Morphological Opening
    # kernel = np.ones((3,3),np.uint8)
    # clean_img = cv.morphologyEx(binary_img, cv.MORPH_OPEN, kernel)

    return binary_img

In [5]:
def whiteBlackRatio(img):
    h = img.shape[0]
    w = img.shape[1]
    #initialized at 1 to avoid division by zero
    blackCount=1
    whiteCount=0
    for y in range(0,h):
        for x in range (0,w):
            if (img[y,x]==0):
                blackCount+=1
            else:
                whiteCount+=1
    return whiteCount/blackCount

In [6]:
def blackPixelsCount(img):
    blackCount=1 #initialized at 1 to avoid division by zero when we calculate the ratios
    h = img.shape[0]
    w = img.shape[1]
    for y in range(0,h):
        for x in range (0,w):
            if (img[y,x]==0):
                blackCount+=1

    return blackCount

In [7]:
def horizontalTransitions(img):
    h = img.shape[0]
    w = img.shape[1]
    maximum=0
    for y in range(0,h):
        prev=img[y,0]
        transitions=0
        for x in range (1,w):
            if (img[y,x]!=prev):
                transitions+=1
                prev= img[y,x]
        maximum= max(maximum,transitions)

    return maximum

In [8]:
def verticalTransitions(img):
    h = img.shape[0]
    w = img.shape[1]
    maximum=0
    for x in range(0,w):
        prev=img[0,x]
        transitions=0
        for y in range (1,h):
            if (img[y,x]!=prev):
                transitions+=1
                prev= img[y,x]
        maximum= max(maximum,transitions)

    return maximum

In [9]:
def histogramAndCenterOfMass(img):
    h = img.shape[0]
    w = img.shape[1]
    histogram=[]
    sumX=0
    sumY=0
    num=0
    for x in range(0,w):
        localHist=0
        for y in range (0,h):
            if(img[y,x]==0):
                sumX+=x
                sumY+=y
                num+=1
                localHist+=1
        histogram.append(localHist)

    return sumX/num , sumY/num, histogram

In [10]:
def number_of_endpoints(img):
    # Apply morphological operations to find endpoints
    kernel = np.array([[1, 1, 1],
                       [1, 10, 1],
                       [1, 1, 1]], dtype=np.uint8)
    dilated_img = cv.dilate(img, kernel)
    endpoints_img = dilated_img - img

    # Count the number of endpoints
    endpoints_count = np.count_nonzero(endpoints_img)

    return endpoints_count

In [11]:
def number_of_loops(img):
    # Apply edge detection to the image
    edges = cv.Canny(img, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv.findContours(edges, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)

    # Count the number of contours with area greater than a threshold (assuming loops will have larger areas)
    loop_count = sum(1 for contour in contours if cv.contourArea(contour) > 100)  # Adjust the threshold as needed

    return loop_count

In [12]:
def number_of_line_crossings(img):
    # Apply Hough Line Transform to detect lines in the image
    lines = cv.HoughLines(img, 1, np.pi/180, 100)

    # Count the number of detected lines
    line_count = len(lines) if lines is not None else 0

    return line_count

In [13]:
def discrete_wavelet_transform(img):
    coeffs = pywt.dwt2(img, 'haar')  # 'haar' is the wavelet family, you can choose another one if needed
    LL, (LH, HL, HH) = coeffs  # LL: Approximation, LH: Horizontal detail, HL: Vertical detail, HH: Diagonal detail
    return LL.flatten(), LH.flatten(), HL.flatten(), HH.flatten()

def discrete_cosine_transform(img):
    return dct(dct(img.T, norm='ortho').T, norm='ortho').flatten()

In [14]:
def fourier_features(img):
    img_resized = cv.resize(img, (32,32), interpolation=cv.INTER_AREA)
    f_transform = np.fft.fft2(img_resized)
    f_shift = np.fft.fftshift(f_transform)
    # Add a small constant to avoid log(0)
    magnitude_spectrum = 20 * np.log(np.abs(f_shift) + 1e-8)  # 1e-8 is a small number to avoid log(0)
    return np.ravel(magnitude_spectrum)

In [15]:
def gradient_orientation_histogram(img):
    gx, gy = np.gradient(img.astype(float))
    magnitude = np.sqrt(gx**2 + gy**2)
    orientation = np.arctan2(gy, gx) * (180 / np.pi) % 180
    histogram, _ = np.histogram(orientation, bins=9, range=(0, 180), weights=magnitude)
    return histogram

In [16]:
def getFeatures(img):
    x,y= img.shape
    featuresList=[]
    # first feature: height/width ratio
    featuresList.append(y/x)
    #second feature is ratio between black and white count pixels
    featuresList.append(whiteBlackRatio(img))
    #third and fourth features are the number of vertical and horizontal transitions
    featuresList.append(horizontalTransitions(img))
    featuresList.append(verticalTransitions(img))

    #print (featuresList)
    #splitting the image into 4 images
    topLeft=img[0:y//2,0:x//2]
    topRight=img[0:y//2,x//2:x]
    bottomeLeft=img[y//2:y,0:x//2]
    bottomRight=img[y//2:y,x//2:x]

    #get white to black ratio in each quarter
    featuresList.append(whiteBlackRatio(topLeft))
    featuresList.append(whiteBlackRatio(topRight))
    featuresList.append(whiteBlackRatio(bottomeLeft))
    featuresList.append(whiteBlackRatio(bottomRight))

    #the next 6 features are:
    #• Black Pixels in Region 1/ Black Pixels in Region 2.
    #• Black Pixels in Region 3/ Black Pixels in Region 4.
    #• Black Pixels in Region 1/ Black Pixels in Region 3.
    #• Black Pixels in Region 2/ Black Pixels in Region 4.
    #• Black Pixels in Region 1/ Black Pixels in Region 4
    #• Black Pixels in Region 2/ Black Pixels in Region 3.
    topLeftCount=blackPixelsCount(topLeft)
    topRightCount=blackPixelsCount(topRight)
    bottomLeftCount=blackPixelsCount(bottomeLeft)
    bottomRightCount=blackPixelsCount(bottomRight)

    featuresList.append(topLeftCount/topRightCount)
    featuresList.append(bottomLeftCount/bottomRightCount)
    featuresList.append(topLeftCount/bottomLeftCount)
    featuresList.append(topRightCount/bottomRightCount)
    featuresList.append(topLeftCount/bottomRightCount)
    featuresList.append(topRightCount/bottomLeftCount)
    #get center of mass and horizontal histogram
    xCenter, yCenter,xHistogram =histogramAndCenterOfMass(img)
    featuresList.append(xCenter)
    featuresList.append(yCenter)
    #featuresList.extend(xHistogram)
    #print(len(featuresList))


    # Structural features
    featuresList.append(number_of_loops(img))
    featuresList.append(number_of_line_crossings(img))
    featuresList.append(number_of_endpoints(img))

    # Transform-based features
    #dwt_features = discrete_wavelet_transform(img)
    #dct_features = discrete_cosine_transform(img)
    #featuresList.append(dwt_features)
    #featuresList.append(dct_features)

    featuresList.extend(gradient_orientation_histogram(img))
    featuresList.extend(fourier_features(img))

    return featuresList

In [17]:
def get_immediate_subdirectories(a_dir):
    return [name for name in os.listdir(a_dir)
        if os.path.isdir(os.path.join(a_dir, name))]

In [18]:
def getListOfFiles(dirName):
    # create a list of file and sub directories
    # names in the given directory
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)

    return allFiles

In [19]:
def write_to_csv(y_test, y_pred, filename):
    with open(filename, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['y_test', 'y_pred'])
        for test, pred in zip(y_test, y_pred):
            writer.writerow([test, pred])

In [20]:
def accuracyy(y_test,y_pred):
    y_test_char=[]
    y_pred_char=[]
    charLabels = ['alif', 'ba', 'ta', 'tha', 'gim', 'ha', 'kha', 'dal' ,'thal', 'ra', 'zay', 'sin', 'shin', 'sad', 'dad', 'tah', 'za', 'ayn', 'gayn', 'fa', 'qaf', 'kaf', 'lam', 'mim', 'non', 'haa', 'waw', 'ya', 'hamza']
    charPositions = ['Beginning', 'Middle', 'End', 'Isolated']
    for test in y_test:
        for i in range(len(charLabels)):
            for j in range(len(charPositions)):
                if test==charLabels[i]+charPositions[j]:
                    y_test_char.append(charLabels[i])
    for test in y_pred:
        for i in range(len(charLabels)):
            for j in range(len(charPositions)):
                if test==charLabels[i]+charPositions[j]:
                    y_pred_char.append(charLabels[i])
    return accuracy_score(y_test_char, y_pred_char)

In [21]:
def trainAndClassify(data, classes, classifiers):
    X_train, X_test, y_train, y_test = train_test_split(data, classes, test_size=0.30)
    accuracies = []

    for idx, clf in enumerate(classifiers):
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        accuracy = accuracyy(y_test, y_pred)
        accuracies.append(accuracy)
        write_to_csv(y_test, y_pred, 'predictions'+str(idx)+'.csv')
        joblib.dump(clf, f'classifier{idx}.pkl')
    return accuracies

In [22]:
def removeMargins(img):
    th, threshed = cv.threshold(img, 245, 255, cv.THRESH_BINARY_INV)
    ## (2) Morph-op to remove noise
    kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (11,11))
    morphed = cv.morphologyEx(threshed, cv.MORPH_CLOSE, kernel)
    ## (3) Find the max-area contour
    cnts = cv.findContours(morphed, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)[-2]
    cnt = sorted(cnts, key=cv.contourArea)[-1]
    ## (4) Crop and save it
    x,y,w,h = cv.boundingRect(cnt)
    dst = img[y:y+h, x:x+w]
    return dst

In [ ]:
import cv2
import os

def analyze_image_sizes(directory):
    dimensions = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(('png', 'jpg', 'jpeg')):
                path = os.path.join(root, file)
                img = cv2.imread(path)
                if img is not None:
                    dimensions.append(img.shape[:2])  # (height, width)
                else:
                    print(f"Warning: Failed to load image {path}")
    return dimensions
directory = './drive/MyDrive/LettersDataset'
image_sizes = analyze_image_sizes(directory)

from collections import Counter

size_counts = Counter(image_sizes)
most_common_size = size_counts.most_common(1)[0][0]
print(f"The most common image size is: {most_common_size}")

average_height = int(sum(dim[0] for dim in image_sizes) / len(image_sizes))
average_width = int(sum(dim[1] for dim in image_sizes) / len(image_sizes))
print(f"The average image size is: {average_height} x {average_width}")


The most common image size is: (32, 32)
The average image size is: 32 x 32


In [34]:
data = []
classes = []
directory = './drive/MyDrive/LettersDataset'
chars = get_immediate_subdirectories(directory)
numOfFeatures = 1052
count=0
charPositions=['Beginning','End','Isolated','Middle']
for char in chars:
    for position in charPositions:
        path = os.path.join(directory, char, position)
        if os.path.isdir(path):
            listOfFiles = getListOfFiles(path)
            for filename in listOfFiles:
                img = cv.imread(filename)
                if img is None:
                    continue
                gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
                cropped = removeMargins(gray_img)
                binary_img = binary_otsus(gray_img, 0)
                features = getFeatures(binary_img)
                if len(features) == numOfFeatures:
                    data.append(features)
                    classes.append(char + position)
                else:
                    print(f"Feature length mismatch in file {filename}, got {len(features)} features")

# Zip the data and classes together
zipped = list(zip(data, classes))
#random.seed(42)  # Use any number to seed the generator

# Shuffle the combined list
random.shuffle(zipped)

# Unzip back into data and classes
data, classes = zip(*zipped)

# Now convert lists to numpy arrays and reshape as necessary
data = np.array(data).reshape(-1, numOfFeatures)
classes = np.array(classes)

In [ ]:
#calssifiers test
classifiers = [ svm.LinearSVC(),
                MLPClassifier(alpha=1e-4, hidden_layer_sizes=(100,), max_iter=1000),
                MLPClassifier(alpha=1e-4, hidden_layer_sizes=(500,200), max_iter=2000),
                MLPClassifier(alpha=5, hidden_layer_sizes=(500, 200,), max_iter=2000),
                MLPClassifier(alpha=50, hidden_layer_sizes=(500, 200,), max_iter=2000),
                MLPClassifier(alpha=20, hidden_layer_sizes=(100,), max_iter=1000),
                MLPClassifier(alpha=0, hidden_layer_sizes=(1000,500), max_iter=1000),
                GaussianNB()]

accuracies = trainAndClassify(data, classes, classifiers)
for idx, accuracy in enumerate(accuracies):
    print(f"Accuracy of Classifier {idx + 1}: {accuracy}")

'''Accuracy of Classifier 1: 0.033926155035762615
Accuracy of Classifier 2: 0.2656098975449449
Accuracy of Classifier 3: 0.2718925188478639
Accuracy of Classifier 4: 0.20722984728397448
Accuracy of Classifier 5: 0.09133964817320704
Accuracy of Classifier 6: 0.11946646046781365
Accuracy of Classifier 7: 0.2678329789290547
Accuracy of Classifier 8: 0.14305045428184807'''

/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:693: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy of Classifier 1: 0.5674418604651162
Accuracy of Classifier 2: 0.1648578811369509
Accuracy of Classifier 3: 0.6211886304909561
Accuracy of Classifier 4: 0.7607235142118863
Accuracy of Classifier 5: 0.5989664082687338
Accuracy of Classifier 6: 0.16434108527131783
Accuracy of Classifier 7: 0.5369509043927648
Accuracy of Classifier 8: 0.6470284237726098


'Accuracy of Classifier 1: 0.033926155035762615\nAccuracy of Classifier 2: 0.2656098975449449\nAccuracy of Classifier 3: 0.2718925188478639\nAccuracy of Classifier 4: 0.20722984728397448\nAccuracy of Classifier 5: 0.09133964817320704\nAccuracy of Classifier 6: 0.11946646046781365\nAccuracy of Classifier 7: 0.2678329789290547\nAccuracy of Classifier 8: 0.14305045428184807'

In [ ]:
#calssifiers test
classifiers = [
    MLPClassifier(alpha=1e-3, hidden_layer_sizes=(50,), max_iter=1000),
    MLPClassifier(alpha=1e-6, hidden_layer_sizes=(300, 200,), max_iter=1000),
]

accuracies = trainAndClassify(data, classes, classifiers)
for idx, accuracy in enumerate(accuracies):
    print(f"Accuracy of Classifier {idx + 1}: {accuracy}")
    '''
Accuracy of Classifier 1: 0.34805722018171276
Accuracy of Classifier 2: 0.33220568335588635'''

Accuracy of Classifier 1: 0.17622739018087855
Accuracy of Classifier 2: 0.765891472868217


In [35]:
#classifiers test
classifiers = [
    SVC(),  # Support Vector Classifier
    KNeighborsClassifier(),  # k-Nearest Neighbors
    DecisionTreeClassifier(),  # Decision Tree
    RandomForestClassifier(),  # Random Forest
]
accuracies = trainAndClassify(data, classes,classifiers)
for idx, accuracy in enumerate(accuracies):
    print(f"Accuracy of Classifier {idx + 1}: {accuracy}")
'''Accuracy of Classifier 1: 0.4087870105062082
Accuracy of Classifier 2: 0.3390639923591213
Accuracy of Classifier 3: 0.6160458452722063
Accuracy of Classifier 4: 0.7163323782234957
Accuracy of Classifier 1: 0.18325923062052968'''

Accuracy of Classifier 1: 0.39749575551782684
Accuracy of Classifier 2: 0.34465195246179964
Accuracy of Classifier 3: 0.5785229202037352
Accuracy of Classifier 4: 0.698641765704584


'Accuracy of Classifier 1: 0.4087870105062082\nAccuracy of Classifier 2: 0.3390639923591213\nAccuracy of Classifier 3: 0.6160458452722063\nAccuracy of Classifier 4: 0.7163323782234957\nAccuracy of Classifier 1: 0.18325923062052968'

In [36]:
classifiers = [
    RandomForestClassifier(),
]

accuracies = trainAndClassify(data, classes,classifiers)
for idx, accuracy in enumerate(accuracies):
    print(f"Accuracy of Classifier {idx + 1}: {accuracy}")

Accuracy of Classifier 1: 0.6865449915110357


In [37]:
#directory of hamza and alif can edit yaa too
#handle spaces bet words
chars = ['ب', 'ت', 'ث', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'غ', 'ف', 'ق','ك', 'ل', 'م', 'ن', 'ه', 'و','ي']
charLabels =['ba', 'ta', 'tha', 'gim', 'ha', 'kha', 'dal' ,'thal', 'ra', 'zay', 'sin', 'shin', 'sad', 'dad', 'tah', 'za', 'ayn', 'gayn', 'fa', 'qaf', 'kaf', 'lam', 'mim', 'non', 'haa', 'waw', 'ya']
positionsLabels=['Beginning','End','Isolated','Middle']
word = ''
def labeltochar(label):
    if label=='alifMiddle'or label=='alifEnd':
        return 'ا'
    if label=='alifBeginning'or label=='alifIsolated':
        return 'أ'
    if label=='hamzaEnd':
        return 'ئ'
    if label=='hamzaBeginning'or label=='hamzaIsolated':
        return 'ء'
    if label=='hamzaMiddle':
        return 'ؤ'

    for i in range(len(charLabels)):
        for j in range(len(positionsLabels)):
            if label==charLabels[i]+positionsLabels[j]:
                return chars[i]

classifier = joblib.load('classifier0.pkl')
folder= getListOfFiles('Test')
count=0
data1 = []
for file in folder:
    img = cv.imread(file)
    img_resized = cv.resize(img, (32,32), interpolation=cv.INTER_AREA)
    gray_img = cv.cvtColor(img_resized, cv.COLOR_BGR2GRAY)
    cropped = removeMargins(gray_img)
    binary_img = binary_otsus(gray_img, 0)
    features = getFeatures(binary_img)
    data1.append(features)
    data2 = np.array(data1).reshape(-1, numOfFeatures)
    prediction = classifier.predict(data2)
    count=count+1
    print(f"Prediction for letter {count}: {prediction[0]}")
    #cv.imshow('image',binary_img)
    char=labeltochar(prediction[0])
    word=word+char
    data1 = []

print(word)
    #increase dataset


Prediction for letter 1: sinIsolated
Prediction for letter 2: yaIsolated
Prediction for letter 3: taIsolated
Prediction for letter 4: thaIsolated
Prediction for letter 5: taIsolated
سيتثت
